In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers
import games_configurations
env_name = 'QuadruppedWalk-v1'#'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#a2c_config = games_configurations.pendulum_lstm_config
#a2c_config = games_configurations.bipedalwalker_lstm_config
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
a2c_config = games_configurations.quadrupped_lstm_config
#a2c_config = a2c_games_configurations.mountain_car_cont_config
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

robot lib not fouund
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-07-01 09:00:58,123	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-01_09-00-58_26348/logs.
2019-07-01 09:00:58,239	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:65259 to respond...
2019-07-01 09:00:58,354	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:15229 to respond...
2019-07-01 09:00:58,363	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-07-01 09:00:58,452	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-01_09-00-58_26348/logs.
2019-07-01 09:00:58,455	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:65259',
 'object_store_address': '/tmp/ray/session_2019-07-01_09-00-58_26348/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-01_09-00-58_26348/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks



agent = A2CAgent(sess,'walker', obs_space, False, action_space, a2c_config)
#agent.restore('nn/walkerBipedalWalker-v2')
agent.train()
#agent.save('nn/latest')

4
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


2019-07-01 09:00:59,838	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=26434) 
(pid=26434) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=26434) For more information, please see:
(pid=26434)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=26434)   * https://github.com/tensorflow/addons
(pid=26434) If you depend on functionality not listed there, please file an issue.
(pid=26434) 
(pid=26434) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=26434)   result = entry_point.load(False)
(pid=26434) robot lib not fouund
(pid=26434) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=26434) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=26436) 
(pid=26436) WARNING: The TensorFlow contrib module will not be included in Tens

(pid=26435) robot lib not fouund
(pid=26435) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=26435) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=26438) robot lib not fouund
(pid=26438) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=26438) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=26435) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=26435)   result = entry_point.load(False)
(pid=26433) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=26433)   result = entry_

Instructions for updating:
Use tf.cast instead.
Frames per seconds:  724.2478066042966
saving next best rewards:  501.628092239641
Frames per seconds:  897.1188575610881
saving next best rewards:  574.2453285507345
Frames per seconds:  900.8414166200948
Frames per seconds:  909.9773035737062
Frames per seconds:  961.7552796736232
Frames per seconds:  1001.5054282532195
Frames per seconds:  1007.5208771292545
Frames per seconds:  1014.5556288689455
Frames per seconds:  998.4298834780448
Frames per seconds:  1006.0725657909416
Frames per seconds:  986.1073285552109
Frames per seconds:  1007.7990740731507
Frames per seconds:  997.465974945647
Frames per seconds:  998.9286254247877
Frames per seconds:  983.4042223966694
Frames per seconds:  986.1455927049927
Frames per seconds:  975.1596997370739
Frames per seconds:  947.4292532424882
saving next best rewards:  638.0203751390982
Frames per seconds:  981.4358220633023
saving next best rewards:  712.6887949579794
Frames per seconds:  978.076

Frames per seconds:  962.3644401054934
Frames per seconds:  937.4546136732671
Frames per seconds:  956.9671321621222
Frames per seconds:  966.7142328203219
Frames per seconds:  955.4859820017996
Frames per seconds:  961.0518296736914
Frames per seconds:  957.1881889209693
saving next best rewards:  12284.296152999985
Frames per seconds:  951.6278129803354
Frames per seconds:  960.3024911039985
Frames per seconds:  964.6888719371386
Frames per seconds:  950.4766988973668
saving next best rewards:  12331.273625328338
Frames per seconds:  960.7869645606164
saving next best rewards:  12351.092437090241
Frames per seconds:  958.6571323266613
Frames per seconds:  952.0579839869056
saving next best rewards:  12411.25988181351
Frames per seconds:  945.9145394393481
saving next best rewards:  12497.571150395577
Frames per seconds:  961.8998632221729
saving next best rewards:  12552.0350483843
Frames per seconds:  952.0727043276911
saving next best rewards:  12567.986806869332
Frames per seconds

Frames per seconds:  947.5810071550081
saving next best rewards:  21186.11997648195
Frames per seconds:  948.014693346365
saving next best rewards:  21242.814804271344
Frames per seconds:  962.2719415776619
saving next best rewards:  21282.207979097482
Frames per seconds:  947.0346281052691
Frames per seconds:  959.7540007745129
saving next best rewards:  21310.05310047325
Frames per seconds:  952.1682129356972
saving next best rewards:  21354.352813755697
Frames per seconds:  948.0410598716217
Frames per seconds:  952.9557013671468
saving next best rewards:  21373.703958390564
Frames per seconds:  928.0729447119833
saving next best rewards:  21406.93483086409
Frames per seconds:  956.465787687586
saving next best rewards:  21412.465068845897
Frames per seconds:  961.2553073810702
saving next best rewards:  21474.282614677482
Frames per seconds:  952.7089628303147
Frames per seconds:  954.3138583314178
saving next best rewards:  21497.540719906872
Frames per seconds:  953.6679906728561

In [ ]:

ray.shutdown()

In [ ]:
import gym
gym.envs.registry.all()